In [1]:
import requests
import json
import pandas as pd
import numpy as np
from pprint import pprint as pp

In [2]:
#Url per info generiche su giochi/generazioni/regioni
url_generations = "https://pokeapi.co/api/v2/generation/"
url_regions = "https://pokeapi.co/api/v2/region/"

#Url utili per gioco competitivo
## Oggetti
url_items = "https://pokeapi.co/api/v2/item/" #Skippa qualche ID
url_item_attributes = "https://pokeapi.co/api/v2/item-attribute/"
url_item_categories = "https://pokeapi.co/api/v2/item-category/"

## Mosse
url_moves = "https://pokeapi.co/api/v2/move/" #Skippa da 826 a 10001
url_move_ailments = "https://pokeapi.co/api/v2/move-ailment/" #Parte da -1
url_move_category = "https://pokeapi.co/api/v2/move-category/" #Parte da 0
url_move_class = "https://pokeapi.co/api/v2/move-damage-class/" 
url_move_target = "https://pokeapi.co/api/v2/move-target/"

## Pokemon
url_pkmn = "https://pokeapi.co/api/v2/pokemon/" #Skippa da 898 a 10001
url_pkmn_species = "https://pokeapi.co/api/v2/pokemon-species/"
url_ability = "https://pokeapi.co/api/v2/ability/" #Skippa da 267 a 10001
url_natures = "https://pokeapi.co/api/v2/nature/" 
url_stats = "https://pokeapi.co/api/v2/stat/"
url_types = "https://pokeapi.co/api/v2/type/" #Skippa da 18 a 10001

In [3]:
def get_data_completeID(url):
    r = requests.get(url)
    count = r.json()['count']
    #print(count)
    i = -1 ###Tiene conto anche di move_ailments e move_category che partono da -1 e 0
    temp_list = []
    while i <= count:
        try:
           # print("Starting iteration n."+str(i))
            r_API = requests.get(url+str(i))
            temp_list.append(r_API.json())
            i += 1
        except json.decoder.JSONDecodeError: ###Gestisce l'assenza degli url -1 e 0 in tutto tranne move_ailments e 
                                             #move_category
            #print("Skip ID")
            i += 1        
    return temp_list

In [4]:
def get_data_to10000(url):
    r = requests.get(url)
    count = r.json()['count']
    #print(count)
    i = 1
    temp_list = []
    while len(temp_list) < count:
        try:
            #print("Starting iteration n."+str(i))
            r_API = requests.get(url+str(i))
            temp_list.append(r_API.json())
            i += 1
        except json.decoder.JSONDecodeError:
            if url == url_items: ###Items skippa degli ID ma non va a 10001
                i += 1
            else:
                i = 10001 ###Imposto iteratore a 10001 per skippare gli ID che farebbero crashare la request        
    return temp_list

In [5]:
#Uso la funzione get_data_completeID per estrarre i dati per tutti quegli URL dove non ho problemi di skip di numeri
gen_list = get_data_completeID(url_generations)
region_list = get_data_completeID(url_regions)
item_attr_list = get_data_completeID(url_item_attributes)
item_categ_list = get_data_completeID(url_item_categories)
move_ailments_list = get_data_completeID(url_move_ailments)
move_category_list = get_data_completeID(url_move_category)
move_class_list = get_data_completeID(url_move_class)
move_target_list = get_data_completeID(url_move_target)
pkmn_spec_list = get_data_completeID(url_pkmn_species)
nature_list = get_data_completeID(url_natures)
stat_list = get_data_completeID(url_stats)

In [6]:
#Uso la funzione get_data_to10000 per gestire i dati degli url che skippano alcuni ID 
item_list = get_data_to10000(url_items)
moves_list = get_data_to10000(url_moves)
pkmn_list = get_data_to10000(url_pkmn)
abilities_list = get_data_to10000(url_ability)
types_list = get_data_to10000(url_types)

Usare metodo json.dump(lista, file_json, indent=) per convertire una lista (o dizionario) in un file json

In [23]:
gen_df = pd.DataFrame(gen_list, columns=["id", "main_region", "moves", "name", "pokemon_species", "types"])
region_df = pd.DataFrame(region_list, columns=["id", "locations", "main_generation", "name"])

items_df = pd.DataFrame(item_list, columns=["attributes", "category", "effect_entries", "id", "name"])
item_attributes_df = pd.DataFrame(item_attr_list, columns=["descriptions", "id", "items", "name"])
item_categories_df = pd.DataFrame(item_categ_list, columns=["id", "items", "name"])

moves_df = pd.DataFrame(moves_list, columns=["accuracy", "damage_class", "generation", "id", "learned_by_pokemon", "name", 
                                             "power", "pp", "type"])
move_ailments_df = pd.DataFrame(move_ailments_list, columns=["id", "moves", "name"])
move_category_df = pd.DataFrame(move_category_list, columns=["descriptions", "id", "moves", "name"])
move_class_df = pd.DataFrame(move_class_list, columns=["descriptions", "id", "moves", "name"])
move_target_df = pd.DataFrame(move_target_list, columns=["descriptions", "id", "moves", "name"])

pkmn_df = pd.DataFrame(pkmn_list, columns=["game_indices", "height", "id", "moves", "name", "weight"])
species_df = pd.DataFrame(pkmn_spec_list, columns=["evolves_from_species", "generation", "has_gender_differences", "id", 
                                                   "is_baby", "is_legendary", "is_mythical", "name", "varieties"])
ability_df = pd.DataFrame(abilities_list, columns=["effect_changes", "effect_entries", "generation", "id", "is_main_series", 
                                                   "name", "pokemon"])
nature_df = pd.DataFrame(nature_list, columns=["decreased_stat", "id", "increased_stat", "name"])
stat_df = pd.DataFrame(stat_list, columns=["affecting_moves", "affecting_natures", "id", "move_damage_class", "name"])
type_df = pd.DataFrame(types_list, columns=["damage_relations", "game_indices", "generation", "id", "name"])

In [24]:
gen_df = gen_df[["id", "name", "main_region", "pokemon_species", "moves", "types"]]
gen_df.columns = ["ID", "Gen_number", "Region_name", "New_pokemon", "New_moves", "New_types"]
region_df = region_df[["id", "name", "main_generation", "locations"]]
region_df.columns = ["ID", "Region_name", "Gen_number", "Locations"]

items_df = items_df[["id", "name", "attributes", "category", "effect_entries"]]
items_df.columns = ["ID", "Name", "Attributes", "Category", "Effect"]
item_attributes_df = item_attributes_df[["id", "name", "descriptions", "items"]]
item_attributes_df.columns = ["ID", "Name", "Descriptions", "Items"]
item_categories_df = item_categories_df[["id", "name", "items"]]
item_categories_df.columns = ["ID", "Name", "Items"]

moves_df = moves_df[["id", "name", "type", "power", "accuracy", "pp", "damage_class", "generation", "learned_by_pokemon"]]
moves_df.columns = ["ID", "Name", "Type", "Power", "Accuracy", "PP", "Damage_class", "Introducted_in", "Learned_by"]
move_ailments_df = move_ailments_df[["id", "name", "moves"]]
move_ailments_df.columns = ["ID", "Name", "Moves"]
move_category_df = move_category_df[["id", "name", "descriptions", "moves"]]
move_category_df.columns = ["ID", "Name", "Descriptions", "Moves"]
move_class_df = move_class_df[["id", "name", "descriptions", "moves"]]
move_class_df.columns = ["ID", "Name", "Descriptions", "Moves"]
move_target_df = move_target_df[["id", "name", "descriptions", "moves"]]
move_target_df.columns = ["ID", "Name", "Descriptions", "Moves"]

pkmn_df = pkmn_df[["id", "game_indices", "name", "moves", "height", "weight"]]
pkmn_df.columns = ["ID", "Game_index", "Name", "Moves", "Height", "Weight"]
species_df = species_df[["id", "name", "varieties", "generation", "evolves_from_species", "has_gender_differences",  "is_baby", 
                         "is_legendary", "is_mythical"]]
species_df.columns = ["ID", "Name", "Varieties", "Generation", "Evolves_from", "Has_gender_diff", "Is_baby", "Is_legendary",
                     "Is_mythical"]
ability_df = ability_df[["id", "name", "generation", "is_main_series", "effect_changes", "effect_entries", "pokemon"]]
ability_df.columns = ["ID", "Name", "Generation", "Is_Main_Series", "Effect_changes", "Effect_entries", "Pokemon"]
nature_df = nature_df[["id", "name", "decreased_stat", "increased_stat"]]
nature_df.columns = ["ID", "Name", "Decreased_stat", "Increased_stat"]
stat_df = stat_df[["id", "name", "affecting_moves", "affecting_natures", "move_damage_class"]]
stat_df.columns = ["ID", "Name", "Affecting_moves", "Affecting_natures", "Move_damage_class"]
type_df = type_df[["id", "game_indices", "name", "generation", "damage_relations"]]
type_df.columns = ["ID", "Game_index", "Name", "Introducted_in", "Damage_relations"]

In [29]:
#Saving .csv
#NON CARICARE SU GITHUB, SONO TROPPO PESANTI

gen_df.to_csv("./API_datasets/gen.csv", index=False)
region_df.to_csv("./API_datasets/region.csv", index=False)
items_df.to_csv("./API_datasets/items.csv", index=False)
item_attributes_df.to_csv("./API_datasets/item_attr.csv", index=False)
item_categories_df.to_csv("./API_datasets/item_cat.csv", index=False)
moves_df.to_csv("./API_datasets/moves.csv", index=False)
move_ailments_df.to_csv("./API_datasets/move_ail.csv", index=False)
move_category_df.to_csv("./API_datasets/move_cat.csv", index=False)
move_class_df.to_csv("./API_datasets/move_class.csv", index=False)
move_target_df.to_csv("./API_datasets/move_target.csv", index=False)
pkmn_df.to_csv("./API_datasets/pokemon.csv", index=False)
species_df.to_csv("./API_datasets/species.csv", index=False)
ability_df.to_csv("./API_datasets/ability.csv", index=False)
nature_df.to_csv("./API_datasets/nature.csv", index=False)
stat_df.to_csv("./API_datasets/stats.csv", index=False)
type_df.to_csv("./API_datasets/types.csv", index=False)